In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
movies = movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [7]:
movies = movies.merge(credits, on ='title')

In [11]:
movies=movies[['movie_id','title','overview','genres','keywords','cast', 'crew']]

In [13]:
movies.dropna(inplace=True)

In [15]:
import ast

def convert (text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [37]:
movies['genres'] = movies['genres'].apply(convert)

In [39]:
movies['keywords'] = movies['keywords'].apply(convert)

In [41]:
def convert_cast (text):
    l=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter<3:
            l.append(i['name'])
        counter +=1
    return l

In [45]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [46]:
def fetch_director(text):
    l=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
            
    return l

In [49]:
 movies['crew'] = movies['crew'].apply(fetch_director)

In [51]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [53]:
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(" ", ""))
    return l

In [55]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)

In [57]:
movies['keywords'] = movies['keywords'].apply(remove_space)

In [61]:
movies['tags']= movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [65]:
new_df = movies[['movie_id','title','tags']]

In [67]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\suman\AppData\Local\Temp\ipykernel_40256\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [69]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\suman\AppData\Local\Temp\ipykernel_40256\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [71]:
import nltk
from nltk.stem import PorterStemmer

In [73]:
ps = PorterStemmer()

In [75]:
def stems (text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [77]:
new_df['tags'] = new_df['tags'].apply(stems)

C:\Users\suman\AppData\Local\Temp\ipykernel_40256\3973021881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stems)


In [79]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [81]:
vector =cv.fit_transform(new_df['tags']).toarray()

In [83]:
from sklearn.metrics.pairwise import  cosine_similarity

In [85]:
similarity = cosine_similarity(vector)

In [87]:
new_df[new_df['title']=='Spider-Man'].index[0]

159

In [91]:
def recommend(movie):
    index = new_df[new_df['title']==movie].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse = True, key = lambda x:x[1])
    for i in distance [1:6]:
        print(new_df.iloc[i[0]].title)

In [93]:
recommend('The Dark Knight Rises')

The Dark Knight
Batman
Batman Begins
Batman
Batman Returns


In [95]:
recommend('Avatar')

Aliens
Silent Running
Mission to Mars
Moonraker
Alien³


In [99]:
import pickle

pickle.dump(new_df, open('model/movies_list.pkl','wb'))
pickle.dump(similarity, open('model/similarity.pkl','wb'))